In [11]:
from pathlib import Path
import os

yr = 2024
d = 16

inp_path = os.path.join(Path(os.path.abspath("")).parents[2], 
             'Input', '{}'.format(yr), 
             '{}.txt'.format(d))

with open(inp_path, 'r') as file:
    inp = file.readlines()

In [12]:
import numpy as np
import heapq
from enum import Enum
from functools import cache

def format_input(inp):
    '''
    Turn the input maze string into a 2D numpy array
    '''
    from itertools import product

    return (np.array([[(1 if c in '.SE' else np.inf) for c in ''.join(l).strip()] for l in inp]), 
            [(i,j) for (i, j) in product(range(len(inp)), range(len(inp[1])-1)) if inp[i][j]=='S'][0], 
            [(i,j) for (i, j) in product(range(len(inp)), range(len(inp[1])-1)) if inp[i][j]=='E'][0]
            )

In [25]:
class Directions(Enum):
    UP = 1
    DOWN = 2
    LEFT = 3
    RIGHT = 4

dir2coord = {Directions.UP: [-1,0],
            Directions.DOWN: [1,0],
            Directions.LEFT: [0,-1],
            Directions.RIGHT: [0,1]}


def turn(dir, left=False):
  d = {Directions.UP: Directions.RIGHT,
       Directions.RIGHT: Directions.DOWN,
       Directions.DOWN: Directions.LEFT,
       Directions.LEFT: Directions.UP}
  if not left:
    return d[dir]
  else:
    return {v:k for k, v in d.items()}[dir]

def arr_to_tuple(arr):
  return tuple([tuple(l) for l in arr])

def tuple_to_arr(arr):
  return np.array([(l) for l in arr])


def manhattan_distance(p1, p2):
  '''
  We use Manhattan Distance from the current location
  to the target as our A* search heuristic

  This simulates a situation where all nodes on a 
  direct path between the current and target nodes
  have an edge cost of 1
  '''
  return np.sum(np.abs(np.array(p1)-np.array(p2)))



class A_Star_Node:
  '''
   A Node which will be utilized for the A* path-finding algorithm
   If different restrictions were placed on the path then we could override the methods in this class
  '''
    
  def __init__(self, loc, direction, num_direction, g, prev, start_loc, end_loc):
    self.loc = loc
    self.direction = direction
    self.num_direction = num_direction
    self.g = g
    self.h = manhattan_distance(loc, end_loc)
    self.f = self.g + self.h
    self.prev = prev
    self.start_loc = start_loc
    self.end_loc = end_loc

  def get_descriptor(self):
    return (self.loc, self.direction, self.num_direction)


  def direction_to_next(self, direction=None):
    if direction is None:
      direction = self.direction
    dloc = dir2coord[direction]
    return tuple(np.add(dloc, list(self.loc)))

  def get_possible_next_descriptors(self):
    '''
    Return the possible next states according to
    the movement specifications
    '''
      
    # If we are at the end location, then return no new locations
    if self.loc == self.end_loc:
      return []

    possible_nexts = [(self.direction_to_next(self.direction), self.direction, 1),
                    (self.loc, turn(self.direction), 1),
                    (self.loc, turn(self.direction, left=True), 1)]


    # If one of the nexts is the end then convert it to the end
    return [n if n[0]!=self.end_loc else (n[0], None, None) for n in possible_nexts]

  def __eq__(self, other):
    if isinstance(other, tuple):
      return self.get_descriptor() == other
    if isinstance(other, A_Star_Node):
      return self.get_descriptor() == other.get_descriptor()
    return None

  def __hash__(self):
    return hash(self.get_descriptor())


def a_star(start, end, arr, A_Star_Node=A_Star_Node, start_dir=Directions.RIGHT):
  '''
  A* Implementation with a priority queue for the queue nodes

  Also implemented hash tables for fast lookups in queue, and searched lists
  '''

  class Heap_Node_Wrapper:
    '''
    Wraps around a heap node so the priority
    queue can order them by their F and H values
    '''
    def __init__(self, node):
      self.node = node

    def __lt__(self, other):
      n = self.node
      other = other.node
      return (n.f, n.h) < (other.f, other.h)

    def __eq__(self, other):
      n = self.node
      other = other.node
      return (n.f, n.h) == (other.f, other.h)

  def in_arr(loc):
    h, w = arr.shape
    return ((0 <= loc[0] and loc[0] < h)
            and (0 <= loc[1] and loc[1] < w))

  s_node = A_Star_Node(start, start_dir, None, arr[start], None, start, end)
  q = [Heap_Node_Wrapper(s_node)]
  qhash = {s_node:s_node} # Mirrors q, just so we can do O(1) existence checks
  searched = {}

  cnt = 0
  while len(q) != 0:

    cur = heapq.heappop(q).node
    del qhash[cur]
    searched[cur]=True

    for n in [x for x in cur.get_possible_next_descriptors() if x not in searched and in_arr(x[0])]:
        
      # n is just a descriptor so if we see a node in the queue matching that
      # descriptor then we can get a reference to it
      n_node = None
      if n in qhash:
        n_node = qhash[n]

      ng = cur.g + arr[n[0]] + (1000 * (cur.direction != n[1]))

      if n_node is None or ng < n_node.g:
        if n_node is None:
          # If we don't have this node in the queue create it
          n_node = A_Star_Node(n[0], n[1], n[2], ng, cur, start, end)
          heapq.heappush(q, Heap_Node_Wrapper(n_node))
          qhash[n_node] = n_node
        else:
          n_node.g = ng
          n_node.prev = cur
    cnt+=1

  # Once we make it to the end node trace back through
  # the path to the beginning
  endns = [s for s in searched if s.loc==end]
  assert(len(endns)==1)
  endn = endns[0]
  path = [endn.get_descriptor()]
  at_beginning = False
  while not at_beginning:
    if endn.prev is not None:
      path.append(endn.prev.get_descriptor())
    endn = endn.prev
    if endn.prev is None:
      at_beginning = True

  while(len(path)>=2 and path[0][0]==end and path[1][0]==end):
    path = path[1:]

  return list(reversed(path))


def print_path(maze, path):
  cost = get_cost(maze, tuple(path))
  if isinstance(maze, tuple):
    maze = tuple_to_arr(maze)
  maze = maze.astype(str)
  maze = np.vectorize(lambda x: '#' if x == 'inf' else '.' if x in ['1.0', '1'] else '#')(maze)
  d2c = {Directions.UP: '^',
         Directions.LEFT: '<',
         Directions.RIGHT: '>',
         Directions.DOWN: 'v'}
  for p in path:
    maze[p[0]] = d2c.get(p[1], '.')
  s = '\n'.join([''.join(l) for l in maze])
  print(s)
  print(f"(Path took {len(path)-1} Steps with a Total Cost of {int(cost)})")

@cache
def get_cost(maze, path):
  '''
  Given a path return, the cost
  of that path
  '''
  if isinstance(path, tuple):
    path = list(path)
  if isinstance(maze, tuple):
    maze = tuple_to_arr(maze)
  c = 0
  path_list = path
  for i, p in enumerate(path_list):
    if i != 0:
      c += maze[p[0]]

      if i>=1 and p[1] != path_list[i-1][1] and p[1] is not None:
        if p[1] in (turn(path_list[i-1][1]), turn(path_list[i-1][1], left=True)):
          c += 999
        else:
          c += 1999
  return int(c)


def find_alternate_paths(path, maze, start, end):
  def find_alternate_paths_(maze, start, end):
    tup_maze = arr_to_tuple(maze)
    min_cost = np.inf
    orig_path = a_star(start, end, maze, A_Star_Node=A_Star_Node)
    min_cost = get_cost(tup_maze, tuple(path))
    def get_potential_alternate_paths(prev_path, path, maze, start, end, min_cost):
      new_potential_paths = []
      if len(prev_path) != 0:
        path = [p for p in path if p not in orig_path]
      for i, path_e in enumerate(path[:-1]):
        for dir in list(Directions):
          potential_next_loc = tuple(np.add(dir2coord[dir], list(path_e[0])))
          if maze[potential_next_loc[0]][potential_next_loc[1]] == 1 and potential_next_loc != end and potential_next_loc not in [e[0] for e in prev_path+path]:
            rest_of_path = ([(path_e[0], dir, 1)] if path_e[1] != dir else []) + a_star(potential_next_loc, end, maze, A_Star_Node=A_Star_Node, start_dir=dir)

            path_up_to = path[:i+1]

            score_rest_of_path = get_cost(tup_maze, tuple(prev_path+path_up_to+rest_of_path))
            if score_rest_of_path < min_cost:
              raise Exception()
            if score_rest_of_path == min_cost:
              new_potential_paths.append(prev_path+path_up_to+rest_of_path)
              new_potential_paths.extend(get_potential_alternate_paths(prev_path+path_up_to, rest_of_path, maze, rest_of_path[0][0], end, min_cost))
      return new_potential_paths
    alternate_paths =  [orig_path] + get_potential_alternate_paths([], orig_path, maze, start, end, min_cost)
    return alternate_paths
  return find_alternate_paths_(maze, start, end)

def print_alternate_paths(maze, alternate_paths):
  from functools import reduce
  locs = set(reduce(lambda a, b : a + b, list(map(lambda l: list(map(lambda x: x[0], l)), alternate_paths))))
  maze = maze.astype(str)
  maze = np.vectorize(lambda x: '#' if x == 'inf' else '.' if x in ['1.0', '1'] else '#')(maze)
  for p in locs:
    maze[p[0]][p[1]] = 'O'
  s = '\n'.join([''.join(l) for l in maze])
  print(s)
  print(f"(There are {int(len(locs))} tiles that are part of best paths)")

In [26]:
import time

t = time.time()

maze, start, end = format_input(inp)

print_path(arr_to_tuple(maze), tuple(a_star(start, end, maze, A_Star_Node=A_Star_Node)))
print_alternate_paths(maze, find_alternate_paths(a_star(start, end, maze, A_Star_Node=A_Star_Node), maze, start, end))

print('\nRUNTIME: ', time.time()-t)

#############################################################################################################################################
#...#.#.........#...#.....#.........#.....#.......#.................#...#.....#.......#.....#.......#.......#...............#.....#.........#
#.#.#.#.#.#.#####.#.#.###.#.#.#####.#.#.#.#.#####.###.#.###########.#.#.#.###.#.#.#####.#.###.###.#.#.#######.#######.#####.#.###.#.#######^#
#.#...#...#.#.....#.#.#.........................#...#.#.....#...#.....#.#...#.#.#.......#.....#...#...#.....#.#...#...#.#...#...#.#.....#.#^#
#.#####.#.#.#.#####.#.#.#.#.###.###.#.#.#.###.#.###.#######.#.#.#########.###.#.###############.#######.###.#.#.#.#.###.#.#####.#.#####.#.#^#
#.......#.#...#...#.#.#.#...#.................#.#.........................#...#.#...#.........#...#.....#...#.#.#.#.....#.#.....#...#.....#^#
#.#.#####.#####.###.#.#.#####.###.#.###.###.###.#########.#.#.###.#.#######.###.#.###.#######.#.#.#.#####.###.###.###.#.#.#.#.#####.###.###^#
#.#...